# Chatbot Tutor de Matemáticas y Física basado en Transformer

**Informe Final — Curso de Profundización en Deep Learning**

Melissa Cardona — Carrera de Física, 2026

### Nota para el profesor

Este informe documenta el diseño, implementación y evaluación de un **Transformer Encoder–Decoder** entrenado desde cero para resolver problemas de matemáticas y física con soluciones paso a paso.

**Para probar el chatbot interactivo**, abra el notebook de demo:

```
notebooks/03_demo_profesor.ipynb
```

El modelo ya viene entrenado en `checkpoints/`. No es necesario reentrenar.

**Instrucciones rápidas:**
1. Clonar o descargar desde: `https://github.com/MelissaCardona2003/Chat-bot-de-matemacticas-y-f-sica-con-TensorFlow.git`
2. Crear entorno: `python -m venv .venv && source .venv/bin/activate && pip install -r requirements.txt`
3. Abrir: `jupyter notebook notebooks/03_demo_profesor.ipynb`
4. Ejecutar todas las celdas — la interfaz Gradio se abrirá automáticamente.

Funciona en **CPU**. No se requiere GPU.

---

# Resumen

Se presenta el diseño, implementación y evaluación de un modelo **Transformer Encoder–Decoder** construido completamente desde cero en TensorFlow, sin utilizar modelos pre-entrenados ni APIs externas. El sistema recibe problemas de matemáticas (aritmética, álgebra) y física (cinemática, dinámica, termodinámica, circuitos eléctricos) y genera soluciones paso a paso.

El modelo (**TransformerV3**) cuenta con **~10,514,849 parámetros**, utiliza tokenización **BPE** (SentencePiece, 4,000 tokens) e incluye una **cabeza de regresión numérica auxiliar** (Answer Head). Fue entrenado sobre un dataset curado de **6,881 problemas** mediante una estrategia innovadora de **tres fases** diseñada para resolver el problema de colapso de cross-attention.

Los resultados muestran una **token accuracy del 73.8%** en validación y un **exact match del 3.0%** (3/100). El logro técnico principal es la **cross-attention selectiva** (entropía normalizada 0.52–0.74), que representa una mejora significativa respecto a versiones anteriores donde la cross-attention estaba completamente colapsada (entropía ≈ 1.0). Esto demuestra que el decoder atiende selectivamente al problema de entrada.

El proyecto pasó por tres versiones iterativas (v1: character-level → v2: BPE → v3: three-phase + answer head) e incluye un pipeline completo: recolección de datos, tokenización BPE, entrenamiento en tres fases con diversity loss, evaluación rigurosa, y despliegue con interfaz interactiva Gradio.

**Palabras clave**: Transformer, Encoder-Decoder, Attention, Cross-Attention, Deep Learning, matemáticas, física, resolución de problemas, TensorFlow, BPE.

---

# 1. Introducción

## 1.1 Motivación

La resolución automática de problemas de matemáticas y física mediante modelos de lenguaje es una de las fronteras más activas de la inteligencia artificial. Sistemas como GPT-4, Gemini y Claude han demostrado capacidades notables en razonamiento matemático, pero operan con cientos de miles de millones de parámetros, infraestructura masiva y datasets propietarios.

Este proyecto nace de una pregunta pedagógica fundamental: **¿qué se puede lograr construyendo un Transformer desde cero, con recursos limitados, para resolver problemas de matemáticas y física?**

La motivación es doble:

1. **Pedagógica**: Implementar un Transformer completo (no solo usar `transformers.AutoModel`) fuerza a comprender cada componente: atención escalada, positional encoding, máscaras causales, residual connections, layer normalization, y el flujo de información entre encoder y decoder.

2. **Científica**: Explorar las limitaciones de un modelo compacto (~10.5M parámetros, tokenización BPE) en tareas que requieren razonamiento numérico, diagnosticar y resolver problemas de entrenamiento como el colapso de cross-attention, y entender *por qué* los LLMs modernos necesitan escalas mucho mayores.

## 1.2 Formulación del problema

El problema se formula como una transformación secuencia-a-secuencia. Dado un problema $x = (x_1, x_2, \dots, x_T)$ en lenguaje natural, el objetivo es generar una solución $y = (y_1, y_2, \dots, y_{T'})$ que contenga los pasos de resolución y la respuesta final.

Formalmente, se busca maximizar la probabilidad condicional:

$$
y^* = \arg\max_y P(y \mid x) = \arg\max_y \prod_{t=1}^{T'} P(y_t \mid y_{<t}, x)
$$

donde cada $y_t$ es un token BPE generado condicionado en el problema completo $x$ (a través del encoder) y en los tokens previamente generados $y_{<t}$ (a través del decoder autoregresivo).

## 1.3 Alcance

El sistema abarca dos dominios:

- **Matemáticas**: Problemas aritméticos y algebraicos de nivel escolar (sumas, restas, multiplicaciones, proporciones, porcentajes) con razonamiento verbal (*word problems*).
- **Física**: Problemas de cinemática, dinámica newtoniana, termodinámica y circuitos eléctricos con aplicación directa de fórmulas.

El formato de salida esperado es:
```
Step 1: Identify the given values...
Step 2: Apply the relevant formula...
Step 3: Calculate...
Answer: [valor numérico]
```

---

# 2. Entorno y Herramientas

## 2.1 Framework de Deep Learning

- **TensorFlow 2.x / Keras**: Framework principal para la implementación del modelo, incluyendo todas las capas (`tf.keras.layers`), el loop de entrenamiento con `GradientTape`, y la gestión de checkpoints.

## 2.2 Cómputo numérico y evaluación

- **NumPy**: Operaciones vectoriales, muestreo probabilístico (top-k sampling), cálculo de métricas.
- **SymPy**: Validación simbólica de respuestas matemáticas (equivalencia algebraica).
- **Matplotlib**: Visualización de curvas de entrenamiento y distribuciones de atención.

## 2.3 Datos

- **HuggingFace Datasets**: Descarga de GSM8K y MATH directamente desde el hub.
- **JSON**: Formato de almacenamiento para todos los datasets procesados.
- **Funciones propias**: Generación paramétrica de problemas de física, conversión de formatos, filtrado de calidad.

## 2.4 Interfaz

- **Gradio 4.x**: Interfaz web interactiva para demostración del chatbot.

## 2.5 Hardware

- **GPU**: NVIDIA RTX 5060 (arquitectura Blackwell, 8 GB VRAM)
- **Nota**: La GPU Blackwell requirió workarounds específicos para XLA y operaciones de cast (ver Sección 5.6).
- **Inferencia**: Funcional en CPU sin modificaciones.

---

# 3. Marco Teórico

## 3.1 La arquitectura Transformer

El Transformer, introducido por Vaswani et al. (2017), reemplaza las redes recurrentes por un mecanismo de **atención puro** que permite procesar secuencias completas en paralelo. La arquitectura consta de un encoder y un decoder, ambos compuestos por capas apiladas de atención y redes feed-forward.

### 3.1.1 Scaled Dot-Product Attention

El mecanismo fundamental es la atención escalada por producto punto. Dadas las matrices de queries $Q$, keys $K$ y values $V$:

$$
\text{Attention}(Q, K, V) = \text{softmax}\!\left(\frac{QK^\top}{\sqrt{d_k}}\right) V
$$

donde $d_k$ es la dimensión de las keys. El factor $\frac{1}{\sqrt{d_k}}$ previene que los productos punto crezcan demasiado para valores grandes de $d_k$, lo cual empujaría al softmax hacia regiones de gradiente pequeño.

### 3.1.2 Multi-Head Attention

En lugar de aplicar una sola función de atención, se proyectan $Q$, $K$ y $V$ en $h$ subespacios diferentes:

$$
\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, \dots, \text{head}_h) W^O
$$

donde cada cabeza se calcula como:

$$
\text{head}_i = \text{Attention}(Q W_i^Q, K W_i^K, V W_i^V)
$$

con $W_i^Q, W_i^K \in \mathbb{R}^{d_{\text{model}} \times d_k}$, $W_i^V \in \mathbb{R}^{d_{\text{model}} \times d_v}$ y $W^O \in \mathbb{R}^{hd_v \times d_{\text{model}}}$.

En nuestra implementación (TransformerV3): $d_{\text{model}} = 256$, $h = 8$, por lo tanto $d_k = d_v = 256/8 = 32$.

### 3.1.3 Positional Encoding

Dado que el Transformer no posee recurrencia ni convoluciones, se inyecta información posicional mediante funciones sinusoidales:

$$
PE_{(pos, 2i)} = \sin\!\left(\frac{pos}{10000^{2i/d_{\text{model}}}}\right)
$$

$$
PE_{(pos, 2i+1)} = \cos\!\left(\frac{pos}{10000^{2i/d_{\text{model}}}}\right)
$$

Estas funciones permiten al modelo inferir posiciones relativas, ya que $PE_{pos+k}$ puede expresarse como transformación lineal de $PE_{pos}$.

### 3.1.4 Estructura del Encoder

Cada capa del encoder consta de:

1. **Multi-Head Self-Attention**: Cada posición atiende a todas las demás posiciones de la entrada.
2. **Add & Layer Normalization**: Conexión residual seguida de normalización.
3. **Feed-Forward Network (FFN)**: Dos capas densas con activación ReLU:

$$
\text{FFN}(x) = \max(0, xW_1 + b_1)W_2 + b_2
$$

con $W_1 \in \mathbb{R}^{d_{\text{model}} \times d_{ff}}$ y $W_2 \in \mathbb{R}^{d_{ff} \times d_{\text{model}}}$. En nuestro caso, $d_{ff} = 1024$.

4. **Add & Layer Normalization**.

Se aplica dropout ($p=0.2$) después de cada sub-capa.

### 3.1.5 Estructura del Decoder

Cada capa del decoder añade una sub-capa adicional:

1. **Masked Multi-Head Self-Attention**: Atención causal que impide que la posición $t$ atienda a posiciones futuras $t' > t$. Se implementa con una máscara *look-ahead*:

$$
M_{ij} = \begin{cases} 0 & \text{si } i \geq j \\ -\infty & \text{si } i < j \end{cases}
$$

2. **Cross-Attention** (Encoder–Decoder): Las queries provienen del decoder, mientras que las keys y values provienen del encoder. Esto permite al decoder "leer" el problema de entrada.

3. **FFN** + Add & Norm (igual que en el encoder).

### 3.1.6 Capa de salida

La salida del decoder pasa por una capa lineal que produce logits sobre el vocabulario:

$$
\text{logits}_t = W_{\text{out}} \cdot h_t^{(L)} + b_{\text{out}} \in \mathbb{R}^{|\mathcal{V}|}
$$

donde $|\mathcal{V}| = 4000$ es el tamaño del vocabulario BPE.

## 3.2 Función de pérdida

Se utiliza entropía cruzada categórica dispersa con **label smoothing** ($\epsilon = 0.1$):

$$
\mathcal{L} = -\sum_{t=1}^{T'} \left[ (1 - \epsilon) \log P(y_t^{\text{true}}) + \frac{\epsilon}{|\mathcal{V}|} \sum_{k} \log P(k) \right] \cdot \mathbb{1}[y_t \neq \text{PAD}]
$$

El label smoothing previene que el modelo se vuelva excesivamente confiado en sus predicciones, actuando como regularizador.

Las posiciones correspondientes al token `<PAD>` se enmascaran para no contribuir a la pérdida.

## 3.3 Learning Rate Schedule

Se utiliza el esquema propuesto en el paper original (Vaswani et al., 2017):

$$
lr = d_{\text{model}}^{-0.5} \cdot \min\!\left(\text{step}^{-0.5}, \; \text{step} \cdot \text{warmup}^{-1.5}\right)
$$

con $\text{warmup\_steps} = 1000$. El learning rate crece linealmente durante los primeros 1000 pasos y luego decae proporcionalmente a $\text{step}^{-0.5}$. Se incluye un factor de escala $s$ para las fases de fine-tuning:

En la Fase 3 se usa $s = 0.1$ para fine-tuning suave.

$$

lr(\text{step}) = s \cdot d_{\text{model}}^{-0.5} \cdot \min(\text{step}^{-0.5}, \text{step} \cdot \text{warmup}^{-1.5})$$

---

# 4. Datasets y Preparación de Datos

## 4.1 Fuentes de datos

La construcción del dataset fue uno de los retos principales del proyecto. Se requieren pares **(problema, solución paso a paso)** donde la solución contenga razonamiento explícito y una respuesta final.

### 4.1.1 GSM8K (Grade School Math 8K)

Dataset de OpenAI (Cobbe et al., 2021) con 8,792 problemas aritméticos verbales de nivel escolar. Cada problema incluye una solución con razonamiento paso a paso y una respuesta final numérica.

- **Conversión**: Se descarga vía HuggingFace y se convierte al esquema unificado del proyecto con `data/convert_gsm8k.py`.
- **Limpieza**: Eliminación de anotaciones tipo `#### ANSWER`, normalización de formato, filtrado por longitud.
- **Resultado**: 8,638 problemas limpios (7,319 train + 1,319 test).

### 4.1.2 MATH Dataset con soluciones LLM

El dataset MATH (Hendrycks et al., 2021) contiene problemas de competencia matemática con soluciones en LaTeX. Dado que las soluciones originales usan notación LaTeX compleja incompatible con tokenización carácter a carácter, se implementó un pipeline alternativo:

1. **Descarga y filtrado** (`data/download_dataset.py`, `data/filter_dataset.py`): Se seleccionaron niveles 1-3 en álgebra, combinatoria y prealgebra.
2. **Generación de soluciones** (`data/generate_math_solutions_llm.py`): Se utilizó un LLM para generar soluciones en texto plano paso a paso, reemplazando el LaTeX por lenguaje natural.
3. **Resultado**: 1,895 problemas con soluciones legibles.

### 4.1.3 Problemas de Física (Generación Paramétrica)

No se encontró un dataset público de física análogo a GSM8K. Se optó por generar problemas de forma paramétrica (`data/generate_physics_templates.py`):

Se definieron templates en 5 áreas:

| Área | Fórmulas | Ejemplo |
|------|----------|---------|
| Cinemática | $v = v_0 + at$, $x = v_0 t + \frac{1}{2}at^2$ | "A car accelerates at 3 m/s² for 5s..." |
| Dinámica | $F = ma$, $W = Fd$ | "A 10 kg box is pushed with 50 N..." |
| Termodinámica | $Q = mc\Delta T$ | "Heat needed for 2 kg water, ΔT=30°C..." |
| Circuitos | $V = IR$, $P = IV$ | "Circuit with 12V and 4Ω..." |
| Caída libre | $h = \frac{1}{2}gt^2$ | "Object dropped from 80 m..." |

Cada template genera problemas con parámetros aleatorios dentro de rangos físicamente razonables, junto con la solución correcta calculada analíticamente.

- **Resultado**: 2,035 problemas de física (2,019 paramétricos + 16 manuales).

## 4.2 Esquema Unificado

Todos los datos se normalizan a un esquema JSON común (`data/schema.py`):

```json
{
  "problem": "A car accelerates from rest at 3 m/s² for 5 seconds...",
  "solution": "Step 1: Identify... Step 2: Apply... Answer: 15 m/s",
  "answer": "15",
  "domain": "physics",
  "source": "physics_templates",
  "topic": "kinematics",
  "split": "train"
}
```

La función de validación verifica que cada entrada tenga los campos requeridos, que `problem` y `solution` no estén vacíos, que la longitud sea razonable, y que el `domain` sea `math` o `physics`.

## 4.3 Dataset Final: Easy Subset

Para la versión v3, se seleccionó un subconjunto curado de problemas de longitud corta/media (`combined_easy.json`) para optimizar el aprendizaje con un modelo compacto:

| | Math | Physics | Total |
|-----|------|---------|-------|
| Train | ~4,800 | ~930 | **5,729** |
| Val | ~420 | ~89 | **509** |
| Test | ~550 | ~93 | **643** |
| **Total** | **~5,770** | **~1,111** | **6,881** |

## 4.4 Tokenización BPE

Se implementó un tokenizador **BPE (Byte Pair Encoding)** usando SentencePiece (`data/subword_tokenizer.py`) con vocabulario de 4,000 tokens:

$$
\mathcal{V} = \{\texttt{<PAD>}, \texttt{<START>}, \texttt{<END>}, \texttt{<UNK>}\} \cup \{\text{subword}_1, \dots, \text{subword}_{3996}\}
$$

El tokenizador BPE se entrena con `split_digits=True` para que cada dígito sea un token individual, y `byte_fallback=True` para manejar caracteres desconocidos.

**Ventaja sobre tokenización carácter a carácter**: Un problema de 100 palabras se tokeniza en ~30-50 tokens BPE, vs ~500 tokens con caracteres. Esto reduce la longitud de secuencia ~5× y permite al modelo operar a nivel de concepto.

## 4.5 Construcción de `tf.data.Dataset`

El `DatasetBuilder` (`data/dataset_builder.py`) construye pipelines eficientes:

1. Carga el JSON del dataset.
2. Tokeniza problemas y soluciones con BPE y padding a longitudes fijas (`max_encoder_len=128`, `max_decoder_len=256`).
3. Extrae el **valor numérico de la respuesta** para el answer head.
4. Construye pares `((encoder_input, decoder_input), (decoder_target, answer_value))`.
5. Aplica shuffle, batching y prefetch.

---

# 5. Implementación

Todo el código fuente se organizó como un paquete Python (`transformer_math_physics_tutor/`) con módulos separados para cada componente.

## 5.1 Multi-Head Attention (`models/multihead_attention.py`)

La implementación sigue fielmente el paper original. La clase `MultiHeadAttention` recibe `d_model` y `num_heads`, crea las proyecciones $W^Q, W^K, W^V, W^O$ como capas `Dense`, y ejecuta:

1. Proyección: $Q' = QW^Q$, $K' = KW^K$, $V' = VW^V$
2. Reshape a `(batch, heads, seq_len, depth)` donde `depth = d_model / num_heads = 32`
3. Scaled dot-product attention con máscara opcional
4. Concatenación de cabezas y proyección de salida

## 5.2 Encoder y Decoder (`models/encoder_layer.py`, `models/decoder_layer.py`)

- **EncoderLayer**: Self-attention → Add & Norm → FFN → Add & Norm
- **DecoderLayer**: Masked self-attention → Add & Norm → Cross-attention → Add & Norm → FFN → Add & Norm

El decoder devuelve además un diccionario de pesos de atención, lo cual permite análisis post-hoc de qué tokens del encoder son más atendidos.

## 5.3 TransformerV3 (`models/transformer_v3.py`)

La clase `TransformerV3` extiende el `Transformer` base añadiendo una **cabeza de regresión numérica** (Answer Head) que predice el valor numérico de la respuesta directamente desde la salida del encoder:

1. **Mean pooling** de la salida del encoder (enmascarando tokens PAD)
2. **MLP de 2 capas**: `Dense(d_model, ReLU) → Dropout(0.1) → Dense(1)`
3. **Huber loss** sobre valores escalados por `ANSWER_SCALE=1000`

Esto fuerza al encoder a codificar información numérica explotable, lo cual mejora la calidad de las representaciones para la cross-attention del decoder.

El forward pass: `model((encoder_input, decoder_input))` → `(logits, answer_pred)` donde logits tiene shape `(batch, seq_len, vocab_size=4000)` y answer_pred tiene shape `(batch,)`.
## 5.4 Loop de Entrenamiento (`training/trainer.py`)
Integra:
Se implementó un `TransformerTrainerV3` con `tf.GradientTape` en lugar de `model.fit()`, lo que permite:
- Positional encoding sinusoidal
- **Loss combinada**: `loss = seq_loss + λ₁ · answer_loss + λ₂ · diversity_loss`
  - `seq_loss`: Cross-entropy con label smoothing
  - `answer_loss`: Huber loss para regresión numérica del answer head
  - `diversity_loss`: Penaliza distribución uniforme de cross-attention (fuerza selectividad)
- **Decoder token masking**: Reemplazo aleatorio del **35%** de los tokens del decoder input por IDs aleatorios. Esto obliga al modelo a usar cross-attention para obtener información del encoder.
- **Gradient clipping** (global norm = 1.0) para estabilidad.
- **Checkpointing** con rotación.
Implementa el esquema del paper original con un factor de escala $s$:
- **Congelamiento selectivo de capas** para entrenamiento por fases.

lr(\text{step}) = s \cdot d_{\text{model}}^{-0.5} \cdot \min(\text{step}^{-0.5}, \text{step} \cdot \text{warmup}^{-1.5})

Implementa el esquema del paper original:
Con `warmup_steps=1000`, el LR máximo alcanza aproximadamente $4.4 \times 10^{-4}$. El factor $s=0.1$ se usa en la Fase 3 (fine-tuning).
$$
lr(\text{step}) = d_{\text{model}}^{-0.5} \cdot \min(\text{step}^{-0.5}, \text{step} \cdot \text{warmup}^{-1.5})
$$

Con `warmup_steps=2000`, el LR máximo alcanza aproximadamente $3.7 \times 10^{-4}$.

## 5.6 Workarounds para GPU Blackwell
3. **XLA Dropout** (`models/xla_dropout.py`): Implementación de dropout compatible con la compilación XLA, usando `tf.random.uniform` en lugar de `tf.nn.dropout`.

4. **`@tf.function` con `input_signature` fijo**: Para evitar re-tracing excesivo en GPUs Blackwell.
La GPU NVIDIA RTX 5060 (basada en la arquitectura Blackwell) presentó problemas con operaciones XLA y `tf.cast`, que fallaban con errores de segmentación. Se implementaron dos workarounds:
Con `warmup_steps=2000`, el LR máximo alcanza aproximadamente $3.7 \times 10^{-4}$.3. **XLA Dropout** (`models/xla_dropout.py`): Implementación de dropout compatible con la compilación XLA, usando `tf.random.stateless_uniform` en lugar de `tf.nn.dropout`.


2. **Cast patch**: Wrapper de `tf.cast` que ejecuta conversiones de tipo en CPU para evitar el bug de Blackwell en modo eager.

1. **XLA flags**: Configuración de `XLA_FLAGS` y `TF_XLA_FLAGS` para apuntar al directorio correcto de CUDA 12.8.
## 5.6 Workarounds para GPU Blackwell1. **XLA flags**: Configuración de `XLA_FLAGS` y `TF_XLA_FLAGS` para apuntar al directorio correcto de CUDA 12.8.

2. **Cast patch**: Wrapper de `tf.cast` que ejecuta conversiones de tipo en CPU para evitar el bug de Blackwell en modo eager.


3. **XLA Dropout** (`models/xla_dropout.py`): Implementación de dropout compatible con la compilación XLA, usando `tf.random.stateless_uniform` en lugar de `tf.nn.dropout`.La GPU NVIDIA RTX 5060 (basada en la arquitectura Blackwell) presentó problemas con operaciones XLA y `tf.cast`, que fallaban con errores de segmentación. Se implementaron dos workarounds:

In [ ]:
# === Configuración del entorno ===
import os, sys, json
import numpy as np

# Ruta del proyecto
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Cargar configuración y training history
checkpoint_dir = os.path.join(project_root, 'checkpoints', 'v3_easy')

with open(os.path.join(checkpoint_dir, 'config.json'), 'r') as f:
    config_dict = json.load(f)

with open(os.path.join(checkpoint_dir, 'training_history.json'), 'r') as f:
    history_raw = json.load(f)
    history = history_raw.get('phase3', history_raw)

with open(os.path.join(checkpoint_dir, 'evaluation_report.json'), 'r') as f:
    eval_report = json.load(f)

print("=" * 60)
print("CONFIGURACION DEL MODELO (TransformerV3)")
print("=" * 60)
for k, v in config_dict.items():
    print(f"  {k:20s}: {v}")
print(f"\n  Tokenización: BPE (SentencePiece, {config_dict.get('vocab_size', 4000)} tokens)")

print(f"  Entrenamiento: 3 fases (P1:30 + P2:100 + P3:50 épocas)")print(f"  Tiempo total fase 3: {history_raw.get('phase3_time_s', 0):.0f}s")

---

# 6. Entrenamiento

## 6.1 Configuración de hiperparámetros

| Hiperparámetro | Valor | Justificación |
|----------------|-------|---------------|
| `d_model` | 256 | Balance entre capacidad y eficiencia |
| `num_heads` | 8 | Estándar para $d_k = 32$ |
| `num_layers` | 4 | Suficiente para capturar patrones sin overfitting |
| `dff` | 1024 | Ratio 4:1 con $d_{\text{model}}$ (como en el paper original) |
| `dropout_rate` | 0.2 | Mayor que el default (0.1) por dataset pequeño |
| `batch_size` | 32 | Limitado por GPU memory |
| `warmup_steps` | 1000 | ~3 épocas de warmup |
| `label_smoothing` | 0.1 | Regularización estándar |
| `decoder_mask_rate` | 0.35 | Forzar uso de cross-attention (35%) |
| `max_encoder_len` | 128 | Optimizado para dataset easy |
| `max_decoder_len` | 256 | Optimizado para dataset easy |
| `vocab_size` | 4,000 | BPE con SentencePiece |

## 6.2 Estrategia de Entrenamiento en Tres Fases

El entrenamiento en tres fases se diseñó para resolver el **colapso de cross-attention** (entropía normalizada ≈ 1.0), donde el decoder ignora completamente la salida del encoder.

### Fase 1 — Pre-entrenamiento del Encoder (30 épocas)
- **Congelar**: Decoder + Final Layer
- **Entrenar**: Encoder + Answer Head
- **Loss**: Solo `answer_loss` (Huber, weight=10.0)
- **Objetivo**: Forzar al encoder a producir representaciones que codifiquen información numérica del problema

### Fase 2 — Entrenamiento del Decoder (100 épocas)
- **Reinicializar**: Pesos de cross-attention (Q, K, V) con Glorot uniform — rompe la simetría colapsada
- **Congelar**: Encoder (preservar representaciones)
- **Entrenar**: Decoder + Final Layer
- **Loss**: `seq_loss + 5·answer_loss + 10·diversity_loss`
- **Decoder masking**: 35%
- **Objetivo**: El decoder aprende a atender selectivamente al encoder

**Observación clave**: La val_accuracy (73.8%) es mayor que la train_accuracy (64.9%). Esto es típico con dropout alto (0.2) y decoder masking agresivo (35%), que penalizan fuertemente el entrenamiento pero no se aplican en validación.

### Fase 3 — Fine-tuning Completo (50 épocas)

- **Descongelar todo**: Todos los parámetros entrenables| Tiempo Fase 3 | ~1,792s (~30 min) |

- **LR reducido**: `lr_scale = 0.1` para ajuste fino| Val Accuracy | **73.8%** |

- **Loss**: `seq_loss + 5·answer_loss + 2·diversity_loss`| Val Loss | 2.383 |

- **Objetivo**: Refinamiento conjunto de toda la red| Train Accuracy | 64.9% |

| Train Loss (final) | 2.915 |

### Resultados del entrenamiento (Fase 3 — última fase):|---------|-------|

| Métrica | Valor |

In [ ]:
# === Curvas de entrenamiento (Fase 3) ===
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

epochs = range(1, len(history.get('train_loss', [])) + 1)

if history.get('train_loss'):
    # Loss
    axes[0].plot(epochs, history['train_loss'], label='Train', linewidth=2, color='#4f46e5')
    axes[0].plot(epochs, history['val_loss'], label='Val', linewidth=2, color='#f97316')
    axes[0].set_title('Loss (Fase 3)', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Época')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # Accuracy
    axes[1].plot(epochs, history['train_accuracy'], label='Train', linewidth=2, color='#4f46e5')
    axes[1].plot(epochs, history['val_accuracy'], label='Val', linewidth=2, color='#f97316')
    axes[1].set_title('Token Accuracy (Fase 3)', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Época')
    axes[1].set_ylabel('Accuracy')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nTrain Acc final: {history.get('train_accuracy',[-1])[-1]:.4f}")
print(f"Val Acc final:   {history.get('val_accuracy',[-1])[-1]:.4f}")
print(f"Val Loss final:  {history.get('val_loss',[-1])[-1]:.4f}")

---

# 7. Inferencia

## 7.1 Generación autoregresiva

Durante entrenamiento se usa **teacher forcing**: el decoder recibe la secuencia objetivo desplazada. Durante inferencia, el decoder opera de forma autoregresiva:

1. Se codifica el problema con el encoder: $H = \text{Encoder}(x)$
2. Se inicializa el decoder con el token `<START>`
3. En cada paso $t$:
   - Se computan los logits: $z_t = \text{Decoder}(y_{<t}, H)$
   - Se selecciona el siguiente token $\hat{y}_t$
   - Se añade al decoder input
4. Se repite hasta generar `<END>` o alcanzar `max_length=300`

## 7.2 Estrategias de decodificación

Se implementaron tres estrategias en `inference/generate.py`:

### Greedy decoding
$$
\hat{y}_t = \arg\max_k z_t[k]
$$

Determinístico pero propenso a generar secuencias repetitivas.

### Top-k sampling
Se restringen los logits a los $k$ tokens más probables, se aplica temperatura $\tau$ y se muestrea:

$$
P(y_t = k) = \frac{\exp(z_t[k] / \tau)}{\sum_{j \in \text{top-}k} \exp(z_t[j] / \tau)}
$$

En la demo se usa $k=10$, $\tau=0.3$.

### Repetition penalty
Para evitar bucles, se penalizan tokens ya generados:

$$
z_t[k] \leftarrow \begin{cases} z_t[k] / \alpha & \text{si } z_t[k] > 0 \text{ y } k \in \text{generados} \\ z_t[k] \cdot \alpha & \text{si } z_t[k] \leq 0 \text{ y } k \in \text{generados} \end{cases}
$$

con $\alpha = 1.3$. Adicionalmente, se implementa detección de n-gram repeat ($n=10$) para terminar anticipadamente si el modelo entra en un bucle.

## 7.3 Métricas en tiempo real

La demo calcula métricas basadas en los logits reales del modelo:

- **Confianza**: $\text{conf} = \frac{1}{T}\sum_{t=1}^{T} \max_v P(v \mid v_{<t})$ — promedio de la probabilidad máxima por paso.
- **Perplexity**: $\text{PPL} = \exp\!\left(-\frac{1}{T}\sum_{t=1}^{T} \log P(y_t \mid y_{<t})\right)$ — sobre los tokens seleccionados.

## 7.4 Interfaz Gradio

Se desarrolló una interfaz Gradio Blocks (`notebooks/03_demo_profesor.ipynb`) con:
- Selector de dominio (math/physics)
- Ejemplos pre-cargados por dominio
- Métricas en tiempo real (confianza, perplexity, tiempo, tokens generados)
- Sección de arquitectura del modelo
- Curvas de entrenamiento
- Resultados de evaluación
- Análisis de limitaciones

---

# 8. Evaluación y Resultados

## 8.1 Métricas utilizadas

Se evaluó el modelo con múltiples familias de métricas:

### Token-level Accuracy
Proporción de tokens BPE correctamente predichos (ignorando padding).

### Exact Match (Answer:)
Se extrae la línea `Answer:` de la predicción y la referencia, y se comparan. Se realiza comparación textual exacta y numérica con tolerancia (±0.5).

### Cross-Attention Entropy

Entropía normalizada de la distribución de cross-attention por capa del decoder. Valores < 0.85 indican atención selectiva.## 8.2 Resultados cuantitativos



### Answer Head MetricsMAE y exact match del answer head de regresión (predicción directa del valor numérico, sin generación secuencial).

In [ ]:
# === Resultados de evaluacion ===
ta = eval_report.get('token_accuracy', {})
em = eval_report.get('exact_match', {})
ce = eval_report.get('cross_attention_entropy', {})
ah = eval_report.get('answer_head', {})

print("=" * 60)
print("RESULTADOS DE EVALUACION (TransformerV3)")
print("=" * 60)
print(f"\n  Token Accuracy (validación): {ta.get('val_acc', 0):.1%}")
print(f"  Token Accuracy (test):       {ta.get('test_acc', 0):.1%}")
print(f"  Val Loss:                    {ta.get('val_loss', 0):.4f}")
print(f"  Test Loss:                   {ta.get('test_loss', 0):.4f}")

print(f"\n  Answer Head MAE:             {ah.get('mae', 0):.1f}")
print(f"  Answer Head Exact (±0.5):   {ah.get('exact_pct', 0):.1f}%")

textual_pct = em.get('textual_pct', 0)
numeric_pct = em.get('numeric_pct', 0)
print(f"\n  Exact Match (textual): {em.get('textual_correct', 0)}/{em.get('textual_total', 0)} = {textual_pct:.1f}%")
print(f"  Exact Match (numérico ±0.5): {em.get('numeric_correct', 0)}/{em.get('numeric_total', 0)} = {numeric_pct:.1f}%")

print(f"\n  Cross-Attention Entropy (normalizada):")

for layer, entropy in sorted(ce.items()):        print(f"    {domain}: {nc}/{nt} = {pct:.1f}%")

    status = 'SELECTIVA' if entropy < 0.85 else 'UNIFORME'        pct = nc / max(nt, 1) * 100

    print(f"    {layer}: {entropy:.3f} — {status}")        nt = stats.get('numeric_total', stats.get('total', 1))

        nc = stats.get('numeric_correct', stats.get('correct', 0))

if em.get('by_domain'):    for domain, stats in sorted(em['by_domain'].items()):
    print(f"\n  Por dominio:")

## 8.3 Análisis de los resultados

### Cross-attention: de colapsada a selectiva

El logro técnico principal de v3 es la **resolución del colapso de cross-attention**. En las versiones v1 y v2, la entropía normalizada de la cross-attention era ≈1.0 (distribución uniforme), lo que significaba que el decoder ignoraba completamente el encoder. Mediante la estrategia de tres fases con reinicialización de cross-attention y diversity loss, se logró reducir la entropía a 0.52–0.74 (selectiva).

### Token accuracy vs exact match

La discrepancia entre 73.8% token accuracy y 3.0% exact match refleja que el modelo aprende el **formato y estructura** de las soluciones (*"Step 1:... Step 2:... Answer:..."*) pero no el **razonamiento numérico** para producir la respuesta correcta.

Con tokenización BPE (vs character-level en v1), la longitud de secuencia se reduce ~5×, permitiendo al modelo operar a nivel de subpalabra. Sin embargo, el modelo aún no puede "copiar" números del problema al resultado — debe generar cada token de la respuesta desde el vocabulario fijo.

### Answer Head

El answer head alcanza 62.2% de exact match (±0.5) en predicción directa del valor numérico. Esto confirma que el **encoder sí codifica información numérica** del problema, pero esta información no se transfiere completamente al decoder durante la generación secuencial.

### Limitación fundamental: ausencia de mecanismo de copia

El Transformer estándar genera tokens de un vocabulario fijo. No tiene la capacidad de "copiar" tokens directamente de la entrada. En tareas donde la respuesta contiene números que aparecen en el problema (e.g., *"A car travels at 60 km/h..."* → *"Answer: 180 km"*), un mecanismo de copia (pointer network) permitiría al decoder copiar selectivamente tokens del encoder cuando sean relevantes.

---

# 9. Limitaciones, Experimentos Fallidos y Trabajo Futuro

## 9.1 Experimento V4: Mecanismo de copia (Pointer-Generator) — Resultado Negativo

### Motivación

Con V3 logrando solo 3% de exact match, la hipótesis era: "el modelo no puede copiar números del problema al output porque genera cada token desde un vocabulario fijo". Se implementó un **Pointer-Generator Network** (See et al. 2017) para permitir al decoder copiar tokens directamente del input del encoder.

### Arquitectura V4 — Qué se implementó

TransformerV4 extiende TransformerV3 con tres componentes adicionales:

1. **Gate p_gen** (`Dense(1, sigmoid, bias_init=1.0)`): Para cada token del decoder, decide si generar del vocabulario (p_gen→1) o copiar del input (p_gen→0). El bias se inicializa en 1.0 para que comience generando.

2. **Distribución de copia**: Se promedian los pesos de cross-attention de las últimas 2 capas del decoder (promediando los 8 heads). Esto produce una distribución de probabilidad sobre los tokens del input que indica "de dónde copiar".

3. **Probabilidades finales blended**:
   ```
   P(token) = p_gen × P_vocab(token) + (1 - p_gen) × P_copy(token)
   ```

**Archivos creados** (todos en local, NO en GitHub):
- `models/transformer_v4.py` — Arquitectura con gate p_gen y copy distribution
- `training/trainer_v4.py` — Trainer con copy_loss (NLL sobre probabilidades) y monitoreo de p_gen
- `inference/generate_v4.py` — Generación autoregresiva con probabilidades blended
- `run_training_v4.py` — Entrenamiento en 3 fases con transfer learning desde V3
- `resume_phase3.py` — Reanudación de Phase 3 tras OOM (batch_size 32→16)
- `evaluation/evaluate_v4.py` — Evaluación completa con análisis de p_gen

### Entrenamiento V4 — Protocolo de 3 fases

| Fase | Épocas | Qué se entrena | Qué se congela | Resultado |
|------|--------|----------------|----------------|-----------|
| **1** | 30 | Encoder + Answer Head + p_gen gate | Decoder | Train Acc=75.3%, p_gen=0.981 |
| **2** | 100 | Decoder + Final Layer + p_gen (cross-attn reinicializada) | Encoder | Train Acc=62.5%, p_gen=0.991 |
| **3** | 50 | TODO desbloqueado (lr_scale=0.1) | Nada | Train Acc=64.5%, p_gen=0.991 |

**Problema técnico en Phase 3**: El entrenamiento fue matado por el OOM killer de Linux (`SIGKILL`, Exit 137) en la época 15/50. Al desbloquear todos los parámetros, el optimizador Adam necesita almacenar estados m (media) y v (varianza) para CADA parámetro, triplicando el uso de memoria. **Solución**: Reducir batch_size de 32 a 16 y reiniciar desde los pesos de Phase 2.

### Resultado: NEGATIVO — El copy mechanism NO funciona

| Métrica | V3 (mejor) | V4 (Pointer-Generator) | Diferencia |
|---|---|---|---|
| Token Acc (val) | 73.8% | 72.5% | **-1.3%** |
| Token Acc (test) | 69.9% | 68.9% | **-1.0%** |
| Exact Match (textual) | 3.0% | 1.0% | **-2.0%** |
| Exact Match (numérico) | 3.5% | 1.4% | **-2.1%** |
| Answer Head MAE | 298.8 | 283.4 | +15.4 (mejor) |
| p_gen promedio | — | 0.9948 | No copia |
| Math exact match | N/A | 0/88 = 0% | — |
| Physics exact match | N/A | 1/12 = 8.3% | — |

**El síntoma más grave**: El modelo V4 genera fórmulas de física para problemas de matemáticas. Por ejemplo:
- Input: *"Natalia has 3 apples and buys 5 more"* → Output: *"Step 1: Use KE = ½mv². Step 2: KE = 0.5 × 10 × 5² = 125 J"*
- Esto indica que la Phase 2 (reinicialización de cross-attention + mecanismo de copia) confundió al decoder, y Phase 3 no recuperó la calidad.

### Análisis detallado: ¿Por qué falló?

**Causa raíz 1: BPE con `split_digits=True` elimina la necesidad de copiar**

El copy mechanism fue diseñado originalmente para summarization (See et al. 2017), donde hay nombres propios y palabras raras que NO están en el vocabulario (OOV). En nuestro caso:
- El tokenizer BPE con `split_digits=True` descompone CUALQUIER número en dígitos individuales: "125" → tokens ["1", "2", "5"]
- Todos los dígitos 0-9 existen en el vocabulario de 4,000 tokens
- **No hay tokens que necesiten copiarse** — el modelo YA puede generar cualquier número

**Causa raíz 2: Sin supervisión explícita para p_gen**

El gate p_gen se entrena solo por backpropagation del loss de generación. No hay una señal explícita que diga "cuando el token target aparece en el input, p_gen debe ser bajo (copiar)". Sin esta señal:
- El gradiente toma el camino de menor resistencia
- Generar desde el vocabulario (p_gen→1.0) es matemáticamente más estable que aprender a focalizar atención en tokens específicos
- p_gen converge a 0.995 en las primeras épocas y nunca baja

**Causa raíz 3: La cross-attention no es suficientemente selectiva para copiar**

La distribución de copia requiere que la cross-attention se concentre en UN token específico del input (el que se quiere copiar). Pero la cross-attention de V3 distribuye atención sobre contexto amplio (entropía 0.52-0.74, no 0.0-0.1 como se necesitaría). Esto hace que la distribución de copia sea "difusa" — apunta a todo y a nada.

**Causa raíz 4: El entrenamiento en 3 fases daña las representaciones V4**

En V3, la Phase 2 reinicializa cross-attention y el modelo se recupera. En V4, el mecanismo de copia depende de una cross-attention selectiva que se destruye al reinicializar. Phase 3 intenta reparar todo simultáneamente (encoder + decoder + copy), pero con lr_scale=0.1 y batch_size=16, no hay suficiente señal de gradiente para que p_gen baje y el copy mechanism se active.

### Lecciones aprendidas — Cuándo usar y NO usar copy mechanism

| Situación | ¿Usar copy? | ¿Por qué? |
|-----------|-------------|-----------|
| Summarization con nombres propios | ✅ SÍ | Nombres no están en vocabulario |
| Traducción con términos técnicos | ✅ SÍ | Términos OOV necesitan copiarse |
| Resolución matemática con BPE | ❌ NO | BPE+split_digits cubre todos los números |
| Modelo con vocab < 1000 | ✅ QUIZÁS | Vocabulario pequeño puede tener OOV |
| Modelo con vocab > 4000 + split_digits | ❌ NO | Todo cubierto por vocabulario |

**Regla general**: El copy mechanism solo aporta valor cuando existen tokens Out-Of-Vocabulary (OOV) frecuentes en el input que deben aparecer en el output. Si el tokenizer ya puede representar todos los tokens relevantes, el copy mechanism es redundante y puede dañar el entrenamiento.

## 9.2 Comparación completa entre versiones

| Aspecto | v1 | v2 | v3 ⭐ | v4 |
|---------|----|----|----|----|
| **Tokenización** | Character (135) | BPE (4000) | BPE (4000) | BPE (4000) |
| **Parámetros** | 7.4M | 10.5M | 10.5M | 10.5M |
| **Dataset** | 12,568 | 6,881 | 6,881 | 6,881 |
| **Token Accuracy** | 82.1% | ~70% | **73.8%** | 68.9% |
| **Exact Match** | 0% | 0% | **3.0%** | 1.0% |
| **Cross-Attention** | Colapsada (1.0) | Colapsada (1.0) | **Selectiva (0.52-0.74)** | — |
| **Answer Head MAE** | — | — | 298.8 | **283.4** |
| **Innovación** | Baseline | BPE | Three-phase training | Copy mechanism (fallido) |
| **Estado** | Descartado | Descartado | **En producción (GitHub)** | Experimental (solo local) |

**V3 es el mejor modelo y el que está en GitHub para evaluación.**

## 9.3 Mejoras propuestas (ordenadas por impacto esperado)

### Alta prioridad (mayor impacto, esfuerzo medio)

1. **Data augmentation numérica**: Generar variaciones de cada problema cambiando los números. Ej: "María tiene 3 manzanas y compra 5 más" → generar 10 variantes con números aleatorios. Expandiría el dataset de ~6,800 a ~60,000+ problemas. El modelo vería más combinaciones numéricas.

2. **Curriculum learning**: Entrenar primero con problemas de 1 paso (suma/resta simple), luego 2 pasos, luego multi-paso. Facilita el aprendizaje gradual de razonamiento aritmético.

### Media prioridad (impacto moderado, esfuerzo alto)

3. **Calculator augmentation**: Agregar un módulo externo que detecte expresiones aritméticas en la salida y las calcule simbólicamente. Separa "qué operación hacer" (Transformer) de "calcular el resultado" (calculadora).

4. **Pre-entrenamiento en corpus general**: Entrenar primero en texto general para desarrollar comprensión lingüística, luego fine-tuning en matemáticas/física.

### Baja prioridad (investigación a largo plazo)

5. **Escalado del modelo**: Aumentar a 50-100M parámetros con significativamente más datos.

6. ~~Mecanismo de copia~~: **Implementado y descartado** (V4). No funciona con BPE+split_digits. NO intentar de nuevo a menos que se cambie a character-level tokenization o vocabulario muy pequeño.

## 9.4 Valor de los resultados negativos

El progreso de v1 a v4 demuestra un proceso científico riguroso:
- **v1 → v2**: Identificar que la tokenización character-level era un cuello de botella → migrar a BPE
- **v2 → v3**: Diagnosticar el colapso de cross-attention → diseñar entrenamiento en tres fases con reinicialización
- **v3 → v4**: Hipótesis sobre mecanismo de copia → implementación completa → resultado negativo documentado → análisis de causas raíz → decisión informada de descartar

Este proceso iterativo de diagnóstico, experimentación y **documentación honesta de resultados negativos** es representativo del trabajo real en investigación de deep learning. Un resultado negativo bien documentado es tan valioso como uno positivo: evita que otros (o yo misma en el futuro) repitan el mismo experimento.

---

# 10. Aprendizajes y Contribuciones Personales

## 10.1 Proceso iterativo — Cuatro versiones del modelo

### Versión 1: Character-level tokenization
- Tokenización carácter a carácter (135 tokens).
- Modelo: 4 capas, d_model=256, 8 heads (~7.4M params).
- Dataset: 12,568 problemas (GSM8K + MATH + física).
- Resultado: 82.1% token accuracy, 0% exact match, cross-attention colapsada.
- **Lección**: La tokenización character-level es un cuello de botella fundamental — el modelo memoriza secuencias de caracteres sin entender la estructura semántica.

### Versión 2: BPE tokenization
- Migración a SentencePiece BPE (4,000 tokens, `split_digits=True`).
- Modelo: ~10.5M params.
- Dataset reducido a 6,881 problemas (easy subset).
- Resultado: ~70% token accuracy, 0% exact match, cross-attention aún colapsada.
- **Lección**: BPE mejora la representación pero no resuelve el colapso de cross-attention. El problema es que el decoder aprende a auto-completar sin leer el encoder.

### Versión 3: Three-phase training + Answer Head ⭐ (Mejor modelo)
- Entrenamiento en tres fases con reinicialización de cross-attention.
- Answer Head para forzar al encoder a codificar información numérica.
- Diversity loss para penalizar cross-attention uniforme.
- Decoder masking agresivo (35%).
- Resultado: 73.8% token accuracy, **3.0% exact match**, **cross-attention selectiva (0.52-0.74)**.
- **Lección**: El diagnóstico profundo y la ingeniería de entrenamiento pueden resolver problemas que parecen arquitectónicos. A veces el problema no es la arquitectura sino cómo se entrena.

### Versión 4: Pointer-Generator Network (Copy Mechanism) — Fallido
- Implementación de mecanismo de copia basado en See et al. 2017.
- Gate p_gen para decidir entre generar del vocabulario o copiar del input.
- Entrenamiento en tres fases con transfer learning desde V3.
- Resultado: 68.9% token accuracy, **1.0% exact match**, p_gen≈0.995 (el modelo NO copia).
- **Lección**: No toda mejora teórica funciona en la práctica. El mecanismo de copia requiere tokens OOV para activarse; con BPE+split_digits, no existen OOV. Más importante: sin supervisión explícita para el gate p_gen, converge al camino de menor resistencia (generar siempre).

## 10.2 Qué aprendí — Guía práctica

### Sobre Transformers
- La implementación from-scratch de Multi-Head Attention requiere manejo cuidadoso de shapes (`[batch, heads, seq_len, depth]`).
- Las máscaras (padding, look-ahead, combined) son el aspecto más delicado del Transformer — un error aquí corrompe silenciosamente todo el entrenamiento.
- El **colapso de cross-attention** es un problema real en modelos encoder-decoder pequeños: el decoder puede aprender a ignorar el encoder si el contexto autoregresivo es suficiente.
- La reinicialización selectiva de pesos es una técnica poderosa para romper simetrías colapsadas.

### Sobre mecanismos de copia — Cuándo SÍ y cuándo NO
- ✅ **SÍ funciona**: Tasks de summarization con nombres propios OOV, traducción con términos técnicos no cubiertos por vocabulario.
- ❌ **NO funciona**: Tasks con tokenización BPE+split_digits que ya cubre todos los tokens relevantes (como nuestro caso).
- ❌ **NO funciona**: Sin supervisión explícita del gate p_gen — el gradiente no tiene incentivo para activar la copia.
- **Clave**: Antes de implementar un copy mechanism, verificar que realmente existen tokens OOV que necesitan copiarse. Si el tokenizer ya puede representarlos, el copy es redundante.

### Sobre datos
- La calidad del dataset importa más que la cantidad — 6,881 problemas fáciles dan mejores resultados que 12,568 mixtos.
- La generación paramétrica de problemas de física es una técnica efectiva para crear datasets balanceados.
- La tokenización BPE con `split_digits=True` es **crítica** para tareas numéricas — permite representar cualquier número como secuencia de dígitos.

### Sobre entrenamiento
- El entrenamiento en fases (congelar/descongelar capas) permite diagnosticar y resolver problemas específicos — es una herramienta de debugging, no solo de optimización.
- La diversity loss es efectiva para forzar cross-attention selectiva.
- El decoder masking al 35% fuerza al modelo a depender del encoder.
- `@tf.function` con `input_signature` fijo es esencial para evitar retracing en GPUs Blackwell (sm_120).
- **OOM en Phase 3**: Al desbloquear todos los parámetros, Adam requiere 3× la memoria del modelo (weights + m + v). Solución: reducir batch_size.

### Sobre GPU y sistemas
- Las GPUs de nueva generación (Blackwell/sm_120) requieren workarounds específicos para XLA, tf.cast y dropout custom.
- `TF_XLA_FLAGS="--tf_xla_auto_jit=2"` es necesario para compilación XLA en Blackwell.
- El OOM killer de Linux (`SIGKILL`, exit code 137) mata procesos sin aviso — monitorear con `dmesg` para diagnosticar.

### Sobre metodología de investigación
- **Formular hipótesis antes de implementar**: "El modelo no copia números → agregar copy mechanism" era una hipótesis razonable pero incorrecta.
- **Verificar supuestos**: Debí verificar PRIMERO si realmente había tokens OOV que necesitaran copiarse (no los había con BPE+split_digits).
- **Documentar resultados negativos**: Un experimento fallido bien documentado evita repetir errores y demuestra madurez científica.
- **Saber cuándo parar**: V4 empeoró respecto a V3 → decisión correcta de revertir a V3 en lugar de seguir iterando sobre una dirección equivocada.

---

# 11. Conclusiones y Estado Actual del Proyecto

## 11.1 Logros principales

Se implementó exitosamente un **Transformer Encoder-Decoder desde cero** en TensorFlow para resolver problemas de matemáticas y física con soluciones paso a paso, con cuatro iteraciones del modelo (v1→v4).

1. **Arquitectura completa from-scratch**: Scaled dot-product attention, multi-head attention, positional encoding sinusoidal, encoder-decoder con máscaras, answer head de regresión — todo implementado sin usar capas Transformer pre-hechas.

2. **Pipeline de datos robusto**: Sistema modular para descargar, convertir, validar y combinar datasets de múltiples fuentes con tokenización BPE (SentencePiece).

3. **Entrenamiento innovador en tres fases**: Estrategia de pre-entrenamiento de encoder → reinicialización de cross-attention → fine-tuning para resolver el colapso de cross-attention, con diversity loss y decoder masking agresivo.

4. **Cross-attention selectiva**: Logro técnico principal — la entropía normalizada pasó de 1.0 (colapsada) a 0.52–0.74 (selectiva), demostrando que el decoder atiende al problema de entrada.

5. **Primeros exact matches**: El modelo V3 logra 3.0% de exact match (vs 0% en v1/v2), confirmando que la arquitectura funciona.

6. **Experimentación con copy mechanism (V4)**: Se implementó un Pointer-Generator Network. El resultado fue negativo (p_gen→1.0, el modelo no copia), pero la documentación rigurosa del fallo y sus causas demuestra madurez científica.

7. **Evaluación rigurosa y honesta**: Se reportan métricas favorables (73.8% token accuracy, 62.2% answer head exact) y desfavorables (3% exact match en generación, copy mechanism fallido), con análisis detallado.

8. **Despliegue funcional**: Interfaz Gradio interactiva con métricas de confianza en tiempo real.

## 11.2 Estado actual del repositorio

### En GitHub (versión para evaluación del profesor)

El repositorio contiene el **modelo V3** — la mejor versión obtenida:

| Componente | Descripción | Ubicación |
|---|---|---|
| Modelo V3 | TransformerV3 con Answer Head, 10.5M params | `models/transformer_v3.py` |
| Pesos entrenados | 3 fases: 30+100+50 épocas | `checkpoints/v3_easy/model_weights.weights.h5` |
| Tokenizer BPE | SentencePiece, 4000 tokens | `checkpoints/v2_subword/sp_tokenizer.model` |
| Evaluación | Token acc + exact match + attention | `evaluation/evaluate.py` |
| Demo Gradio | Interfaz interactiva | `notebooks/03_demo_profesor.ipynb` |
| Entrenamiento | Script completo 3 fases | `run_training.py` |

### En local (NO en GitHub) — Archivos experimentales V4

Estos archivos se mantienen localmente como referencia del experimento V4:

| Archivo | Descripción |
|---|---|
| `models/transformer_v4.py` | Arquitectura con gate p_gen + copy distribution |
| `training/trainer_v4.py` | Trainer con copy_loss y monitoreo de p_gen |
| `inference/generate_v4.py` | Generación autoregresiva con probabilidades blended |
| `run_training_v4.py` | Entrenamiento 3 fases V4 (batch_size=32) |
| `resume_phase3.py` | Reanudación Phase 3 tras OOM (batch_size=16) |
| `evaluation/evaluate_v4.py` | Evaluación V4 con análisis p_gen |
| `checkpoints/v4_copy/` | Pesos, configs, historial de entrenamiento V4 |

## 11.3 Conclusión principal

El valor del proyecto reside en:
- **(a)** La **implementación completa from-scratch** de un pipeline de deep learning end-to-end
- **(b)** El **proceso científico iterativo** de diagnóstico y solución (v1→v2→v3→v4)
- **(c)** La resolución del **colapso de cross-attention** mediante ingeniería de entrenamiento
- **(d)** La **documentación honesta de resultados negativos** (V4 copy mechanism)

El proyecto demuestra que un Transformer de ~10.5M parámetros con tokenización BPE y entrenamiento en tres fases puede:
- Aprender el **formato y estilo** de soluciones matemáticas y físicas (73.8% token accuracy)
- Lograr **cross-attention selectiva** (un requisito fundamental para que el decoder "lea" el problema)
- Producir algunos **exact matches** (3%) — limitados por la capacidad de razonamiento aritmético del modelo

La limitación fundamental no es la ausencia de mecanismo de copia (como se demostró con V4), sino la **capacidad inherente de un Transformer estándar de 10.5M parámetros para realizar razonamiento aritmético exacto**.

## 11.4 Roadmap — Cómo continuar mejorando el proyecto

Si se desea retomar el proyecto en el futuro, estas son las acciones recomendadas en orden de prioridad:

### Paso 1: Data Augmentation numérica (1-2 días de trabajo)
```
Objetivo: Expandir dataset de 6,881 → 60,000+ problemas
Cómo: Para cada problema, generar N variantes cambiando los valores numéricos
Ejemplo: "María tiene 3 manzanas" → "María tiene 7 manzanas", "María tiene 12 manzanas"...
Script: Crear data/augment_numeric.py
Entrenar: Mismo run_training.py con más datos
Impacto esperado: Mejorar exact match de 3% a 8-15%
```

### Paso 2: Curriculum Learning (1 día adicional)
```
Objetivo: Entrenar primero con problemas fáciles, luego difíciles
Cómo: Ordenar problemas por número de pasos (1 paso → 2 pasos → multi-paso)
Modificar: training/trainer.py para cambiar el dataset entre fases
Impacto esperado: Aprendizaje más estable, mejores representaciones internas
```

### Paso 3: Calculator Augmentation (2-3 días)
```
Objetivo: Delegar cálculos aritméticos a un módulo simbólico
Cómo: Post-procesar la salida del modelo, detectar expresiones como "15 × 6 ="
       y reemplazar con el resultado correcto usando sympy o eval seguro
Script: Crear inference/calculator.py
Impacto esperado: Exact match podría subir a 20-40% si el razonamiento es correcto
```

### Qué NO intentar de nuevo
- ❌ **Copy mechanism (Pointer-Generator)**: Ya probado, no funciona con BPE+split_digits
- ❌ **Aumentar épocas sin más datos**: El modelo ya converge; más épocas = overfitting
- ❌ **Reducir dropout**: El modelo ya tiene 0.2; reducirlo aumenta overfitting sin mejorar exact match

### Para retomar el proyecto
1. Activar el entorno: `source .venv/bin/activate`
2. Verificar GPU: `python -c "import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))"`
3. Ejecutar evaluación V3: `python evaluation/evaluate.py`
4. Ejecutar demo: Abrir `notebooks/03_demo_profesor.ipynb` y ejecutar todas las celdas
5. Ver archivos V4 (referencia): `ls models/transformer_v4.py training/trainer_v4.py`

---

# 12. Referencias

1. Vaswani, A., Shazeer, N., Parmar, N., Uszkoreit, J., Jones, L., Gomez, A. N., Kaiser, L., & Polosukhin, I. (2017). **Attention Is All You Need.** *Advances in Neural Information Processing Systems, 30* (NeurIPS 2017). https://arxiv.org/abs/1706.03762

2. Cobbe, K., Kosaraju, V., Bavarian, M., Chen, M., Jun, H., Kaiser, L., Plappert, M., Tworek, J., Hilton, J., Nakano, R., Hesse, C., & Schulman, J. (2021). **Training Verifiers to Solve Math Word Problems.** https://arxiv.org/abs/2110.14168

3. Hendrycks, D., Burns, C., Kadavath, S., Arora, A., Basart, S., Tang, E., Song, D., & Steinhardt, J. (2021). **Measuring Mathematical Problem Solving With the MATH Dataset.** *Advances in Neural Information Processing Systems, 34* (NeurIPS 2021). https://arxiv.org/abs/2103.03874

4. Radford, A., Wu, J., Child, R., Luan, D., Amodei, D., & Sutskever, I. (2019). **Language Models are Unsupervised Multitask Learners.** OpenAI.

5. Brown, T. B., et al. (2020). **Language Models are Few-Shot Learners.** *Advances in Neural Information Processing Systems, 33* (NeurIPS 2020). https://arxiv.org/abs/2005.14165

6. Wei, J., et al. (2022). **Chain-of-Thought Prompting Elicits Reasoning in Large Language Models.** *Advances in Neural Information Processing Systems, 35* (NeurIPS 2022). https://arxiv.org/abs/2201.11903

7. Ba, J. L., Kiros, J. R., & Hinton, G. E. (2016). **Layer Normalization.** https://arxiv.org/abs/1607.06450

8. He, K., Zhang, X., Ren, S., & Sun, J. (2016). **Deep Residual Learning for Image Recognition.** *CVPR 2016*. https://arxiv.org/abs/1512.03385

9. Sennrich, R., Haddow, B., & Birch, A. (2016). **Neural Machine Translation of Rare Words with Subword Units.** *ACL 2016*. https://arxiv.org/abs/1508.07909

10. Kingma, D. P., & Ba, J. (2015). **Adam: A Method for Stochastic Optimization.** *ICLR 2015*. https://arxiv.org/abs/1412.6980

11. See, A., Liu, P. J., & Manning, C. D. (2017). **Get To The Point: Summarization with Pointer-Generator Networks.** *ACL 2017*. https://arxiv.org/abs/1704.04368

12. Vinyals, O., Fortunato, M., & Jaitly, N. (2015). **Pointer Networks.** *NeurIPS 2015*. https://arxiv.org/abs/1506.03134